In [1]:
train_url = 'https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/main/data/customer_train.csv'
test_url  = 'https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/main/data/customer_test.csv'

import pandas as pd

train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

In [21]:
#전처리

# 불필요한 컬럼 제거
# 결측치 처리
# 인코딩
# 스케일링
# 더미화
# 독립변수, 종속변수 분리
# 학습, 훈련 데이터 분할
  # train, test

In [2]:
train.info()
train.head()
print(train.isnull().sum())
print(train.select_dtypes(include='object').columns)
print(train.nunique()) # -> 일단 작으면 질적변수 아닐까? 생각해야함
# 수치형 변수 
# 범주형 변수 - 주구매상품, 주구매지점 -> 인코딩 or 더미화를 해야함

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   회원ID     3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   방문일수     3500 non-null   int64  
 7   방문당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
 10  성별       3500 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 300.9+ KB
회원ID          0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
방문일수          0
방문당구매건수       0
주말방문비율        0
구매주기          0
성별            0
dtype: int64
Index(['주구매상품', '주구매지점'], dtype='object')
회원ID       3500
총구매액       3396
최대구매액      2576
환불금액       1114
주구매상품        42
주구매지점        24
방문일수    

In [5]:
##불필요한 컬럼 제거
#회원ID, 환불금액은 null값이 너무 많아서 예측하는데 의미 없음
train.주구매지점.value_counts()
train.주구매지점.nunique()

24

In [7]:
train.select_dtypes(exclude=['object']).describe().T[['mean', 'min', '50%', 'max']]

,mean,min,50%,max
회원ID,1.749500e+03,0.0,1.749500e+03,3.499000e+03
총구매액,9.191925e+07,-52421520.0,2.822270e+07,2.323180e+09
최대구매액,1.966424e+07,-2992000.0,9.837000e+06,7.066290e+08
환불금액,2.407822e+07,5600.0,7.392000e+06,5.637530e+08
방문일수,1.925371e+01,1.0,8.000000e+00,2.850000e+02
방문당구매건수,2.834963e+00,1.0,2.333333e+00,2.208333e+01
주말방문비율,3.072463e-01,0.0,2.564103e-01,1.000000e+00
구매주기,2.095829e+01,0.0,1.300000e+01,1.660000e+02
성별,3.760000e-01,0.0,0.000000e+00,1.000000e+00


In [9]:
set(train['주구매상품'].unique()) - set(test['주구매상품'].unique()) #train이 더 많네 그럼 상관 No

{'소형가전'}

In [212]:
train

,회원ID,총구매액,최대구매액,환불금액,주구매상품,주구매지점,방문일수,방문당구매건수,주말방문비율,구매주기,성별
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17,0
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1,0
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16,1
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85,0
...,...,...,...,...,...,...,...,...,...,...,...
3495,3495,3175200,3042900,NaN,골프,본 점,1,2.000000,1.000000,0,1
3496,3496,29628600,7200000,6049600.0,시티웨어,부산본점,8,1.625000,0.461538,40,1
3497,3497,75000,75000,NaN,주방용품,창원점,1,1.000000,0.000000,0,0
3498,3498,1875000,1000000,NaN,화장품,본 점,2,1.000000,0.000000,39,0


In [11]:
remove_col = ['회원ID', '환불금액']
train_preprocessing = train.copy()
train_preprocessing = train_preprocessing.drop(columns=remove_col)
train_preprocessing

,총구매액,최대구매액,주구매상품,주구매지점,방문일수,방문당구매건수,주말방문비율,구매주기,성별
0,68282840,11264000,기타,강남점,19,3.894737,0.527027,17,0
1,2136000,2136000,스포츠,잠실점,2,1.500000,0.000000,1,0
2,3197000,1639000,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,16077620,4935000,기타,광주점,18,2.444444,0.318182,16,1
4,29050000,24000000,보석,본 점,2,1.500000,0.000000,85,0
...,...,...,...,...,...,...,...,...,...
3495,3175200,3042900,골프,본 점,1,2.000000,1.000000,0,1
3496,29628600,7200000,시티웨어,부산본점,8,1.625000,0.461538,40,1
3497,75000,75000,주방용품,창원점,1,1.000000,0.000000,0,0
3498,1875000,1000000,화장품,본 점,2,1.000000,0.000000,39,0


In [ ]:
# 여긴 결측치 채울거 없지만 있다면
train_preprocessing['fillna_col'] = train_preprocessing['fillna_col'].fillna(train_preprocessing['fillna_col'].mean())
train_preprocessing['fillna_col_str'] = train_preprocessing['fillna_col_str'].fillna(train_preprocessing['fillna_col_str'].value_counts().index[0])

In [53]:
# 인코딩 
# get_dummies 간단하게 범주화를 원핫인코딩
# LabelEncoder 사용해서 컬럼수는 늘리지 않고 인코딩

# pd.get_dummies(train_preprocessing) #col값이 너무 많아짐

,총구매액,최대구매액,방문일수,방문당구매건수,주말방문비율,구매주기,성별,주구매상품_가공식품,주구매상품_가구,주구매상품_건강식품,...,주구매지점_안양점,주구매지점_영등포점,주구매지점_울산점,주구매지점_인천점,주구매지점_일산점,주구매지점_잠실점,주구매지점_전주점,주구매지점_창원점,주구매지점_청량리점,주구매지점_포항점
0,68282840,11264000,19,3.894737,0.527027,17,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2136000,2136000,2,1.500000,0.000000,1,0,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,3197000,1639000,2,2.000000,0.000000,1,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,16077620,4935000,18,2.444444,0.318182,16,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,29050000,24000000,2,1.500000,0.000000,85,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3175200,3042900,1,2.000000,1.000000,0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3496,29628600,7200000,8,1.625000,0.461538,40,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3497,75000,75000,1,1.000000,0.000000,0,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3498,1875000,1000000,2,1.000000,0.000000,39,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
# 라벨링을 하되 col수를 증가 시키지 않을 수 있음
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_preprocessing['주구매상품'])
le_predict = le.transform(train_preprocessing['주구매상품'])
train_preprocessing['주구매상품'] = le_predict

le2 = LabelEncoder()
le2.fit(train_preprocessing['주구매지점'])
le2_predict = le2.transform(train_preprocessing['주구매지점'])
train_preprocessing['주구매지점'] = le2_predict

In [15]:
train_preprocessing

,총구매액,최대구매액,주구매상품,주구매지점,방문일수,방문당구매건수,주말방문비율,구매주기,성별
0,68282840,11264000,5,0,19,3.894737,0.527027,17,0
1,2136000,2136000,21,19,2,1.500000,0.000000,1,0
2,3197000,1639000,6,1,2,2.000000,0.000000,1,1
3,16077620,4935000,5,2,18,2.444444,0.318182,16,1
4,29050000,24000000,15,8,2,1.500000,0.000000,85,0
...,...,...,...,...,...,...,...,...,...
3495,3175200,3042900,3,8,1,2.000000,1.000000,0,1
3496,29628600,7200000,22,9,8,1.625000,0.461538,40,1
3497,75000,75000,32,21,1,1.000000,0.000000,0,0
3498,1875000,1000000,41,8,2,1.000000,0.000000,39,0


In [17]:
from sklearn.preprocessing import StandardScaler
scaled_df = train_preprocessing[['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']]

sc = StandardScaler()
sc.fit(scaled_df)

transform_data = sc.transform(scaled_df)
train_preprocessing.loc[:, ['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data

/var/folders/w9/d2ssl_k51sv6ns459qn6lhk00000gn/T/ipykernel_30711/595695857.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.14458009 -0.54918957 -0.5426996  ... -0.56179637 -0.55078606
  1.04709431]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_preprocessing.loc[:, ['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data
/var/folders/w9/d2ssl_k51sv6ns459qn6lhk00000gn/T/ipykernel_30711/595695857.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.26260786 -0.54796686 -0.56350404 ... -0.61239772 -0.58348042
  0.46792117]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_preprocessing.loc[:, ['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data
/var/folders/w9/d2ssl_k51sv6ns459qn6lhk00000gn/T/ipykernel_30711/595695857.py:8: Fut

In [19]:
train_preprocessing

,총구매액,최대구매액,주구매상품,주구매지점,방문일수,방문당구매건수,주말방문비율,구매주기,성별
0,-0.144580,-0.262608,5,0,-0.009338,0.554247,0.527027,-0.159962,0
1,-0.549190,-0.547967,21,19,-0.635003,-0.698168,0.000000,-0.806554,0
2,-0.542700,-0.563504,6,1,-0.635003,-0.436675,0.000000,-0.806554,1
3,-0.463911,-0.460465,5,2,-0.046142,-0.204236,0.318182,-0.200374,1
4,-0.384561,0.135544,15,8,-0.635003,-0.698168,0.000000,2.588052,0
...,...,...,...,...,...,...,...,...,...
3495,-0.542833,-0.519615,3,8,-0.671807,-0.436675,1.000000,-0.846966,1
3496,-0.381022,-0.389656,22,9,-0.414180,-0.632794,0.461538,0.769513,1
3497,-0.561796,-0.612398,32,21,-0.671807,-0.959661,0.000000,-0.846966,0
3498,-0.550786,-0.583480,41,8,-0.635003,-0.959661,0.000000,0.729101,0


In [21]:
y = train_preprocessing['성별']
x = train_preprocessing.drop(columns = ['성별'])

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y) #stratify 분류에서만 사용

In [101]:
# 모델링

# 회귀, 분류 확인하여 모델 선언
# 학습데이터로 모델 학습
## (option) 그리드 서치로 최적 파라미터 / 안해도 무관
## 랜포가 짱
# 평가데이터에 대해 모델 예측

성별
0    2184
1    1316
Name: count, dtype: int64

In [25]:
#만약 시험장에 들어갔는데 모델링이 생각이 안 난다??????
import sklearn
print(sklearn.__all__)

['calibration', 'cluster', 'covariance', 'cross_decomposition', 'datasets', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'experimental', 'externals', 'feature_extraction', 'feature_selection', 'gaussian_process', 'inspection', 'isotonic', 'kernel_approximation', 'kernel_ridge', 'linear_model', 'manifold', 'metrics', 'mixture', 'model_selection', 'multiclass', 'multioutput', 'naive_bayes', 'neighbors', 'neural_network', 'pipeline', 'preprocessing', 'random_projection', 'semi_supervised', 'svm', 'tree', 'discriminant_analysis', 'impute', 'compose', 'clone', 'get_config', 'set_config', 'config_context', 'show_versions']


In [109]:
import sklearn.ensemble
print(dir(sklearn.ensemble))

['AdaBoostClassifier', 'AdaBoostRegressor', 'BaggingClassifier', 'BaggingRegressor', 'BaseEnsemble', 'ExtraTreesClassifier', 'ExtraTreesRegressor', 'GradientBoostingClassifier', 'GradientBoostingRegressor', 'HistGradientBoostingClassifier', 'HistGradientBoostingRegressor', 'IsolationForest', 'RandomForestClassifier', 'RandomForestRegressor', 'RandomTreesEmbedding', 'StackingClassifier', 'StackingRegressor', 'VotingClassifier', 'VotingRegressor', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_bagging', '_base', '_forest', '_gb', '_gradient_boosting', '_hist_gradient_boosting', '_iforest', '_stacking', '_voting', '_weight_boosting']


In [27]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier # 지금 데이터는 분류니깐 classifier / RandomForestRegressor

rf = RandomForestClassifier(random_state =1)
rf.fit(X_train, y_train)

val_predict = rf.predict(X_test)
val_predict_train = rf.predict(X_train) 
val_rf_prob = rf.predict_proba(X_test)

In [29]:
val_rf_prob

array([[0.61, 0.39],
       [0.59, 0.41],
       [0.55, 0.45],
       ...,
       [0.83, 0.17],
       [0.78, 0.22],
       [0.66, 0.34]])

In [31]:
# XGBoost extreme gradient boosting
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=120)
xgb.fit(X_train, y_train)
val_xgb_predict = xgb.predict(X_test) 
val_xgb_prob = xgb.predict_proba(X_test)

In [33]:
# 평가
from sklearn.metrics import roc_auc_score, accuracy_score
accuracy_score(y_test, val_predict)

0.6104761904761905

In [35]:
accuracy_score(y_train,val_predict_train)

0.9995918367346939

In [236]:
accuracy_score(y_test, val_xgb_predict)

0.5885714285714285

In [238]:
#roc_auc는 prob가 필요함
roc_auc_score(y_test, val_rf_prob[: ,1])

0.5968711952845684

In [240]:
roc_auc_score(y_test, val_xgb_prob[:,1])

0.5898424968595999

In [37]:
#set(train['주구매상품'].unique()) - set(test['주구매상품'].unique())
set(test['주구매상품'].unique()) - set(train_preprocessing['주구매상품'].unique())

{'가공식품',
 '가구',
 '건강식품',
 '골프',
 '구두',
 '기타',
 '남성 캐주얼',
 '남성 트랜디',
 '남성정장',
 '농산물',
 '대형가전',
 '디자이너',
 '란제리/내의',
 '명품',
 '모피/피혁',
 '보석',
 '생활잡화',
 '섬유잡화',
 '셔츠',
 '수산품',
 '스포츠',
 '시티웨어',
 '식기',
 '아동',
 '악기',
 '액세서리',
 '육류',
 '일용잡화',
 '젓갈/반찬',
 '주류',
 '주방가전',
 '주방용품',
 '차/커피',
 '축산가공',
 '침구/수예',
 '캐주얼',
 '커리어',
 '통신/컴퓨터',
 '트래디셔널',
 '피혁잡화',
 '화장품'}

In [244]:
# test 테스트 데이터에 대해서도 평가를 해야함
# test 데이터를 train 데이터의 전처리 과정과 동일하게 전처리 해줌

In [45]:
test_preprocessing = test.copy()
test_preprocessing = test_preprocessing.drop(columns=remove_col)

#그냥 원래 학습시켜놓은거로 사용하면 됨.
le_predict_test = le.transform(test_preprocessing['주구매상품'])
le2_predict_test = le2.transform(test_preprocessing['주구매지점'])

test_preprocessing['주구매상품'] = le_predict_test
test_preprocessing['주구매지점'] = le2_predict_test

scaled_df_test = test_preprocessing[['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']]
transform_data_test = sc.transform(scaled_df_test)
test_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data_test

/var/folders/w9/d2ssl_k51sv6ns459qn6lhk00000gn/T/ipykernel_30711/1876353695.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.1285689   1.3372254   1.30499603 ...  1.02814582 -0.01790846
 -0.55844006]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data_test
/var/folders/w9/d2ssl_k51sv6ns459qn6lhk00000gn/T/ipykernel_30711/1876353695.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.07302034  0.59065555 -0.15128409 ...  0.19025261 -0.048276
 -0.60820862]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data_test
/var/folders/w9/d2ssl_k51sv6ns459qn6lhk00000gn/T/ipykernel_30711/1876353695.

In [51]:
rf_test_predict = rf.predict(test_preprocessing)
rf_test_predict_proba = rf.predict_proba(test_preprocessing)[:,1]
xgb_test_predict = xgb.predict(test_preprocessing)
xgb_test_predict_proba = xgb.predict_proba(test_preprocessing[:,1])


InvalidIndexError: (slice(None, None, None), 1)

In [59]:
predict_df = pd.DataFrame(rf_test_predict, columns=['pred'])

In [65]:
predict_df.to_csv('result.csv',index=False)

In [67]:
pd.read_csv('result.csv')

,pred
0,1
1,0
2,1
3,0
4,0
...,...
2477,1
2478,0
2479,1
2480,0
